<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/BegginingOfGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [105]:
import ipywidgets as widgets
from IPython.display import display

from firebase import firebase

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

def createWelcomeScreen():
    # Create main title widget
    title_widget = widgets.HTML("<h1>Welcome to the Python Game</h1>")

    # Create second title widget
    subtitle_widget = widgets.HTML("<h2>It's like Squid Game but here you stay alive :)</h2>")

    def handle_manager_button_click(b):
      screen_container.close()
      display(createPopup())

    def handle_player_button_click(b):
        screen_container.close()
        display(createGameScreen())

    # Create manager button widget
    manager_button = widgets.Button(description="Manager")
    manager_button.on_click(handle_manager_button_click)

    # Create player button widget
    player_button = widgets.Button(description="Player")
    player_button.on_click(handle_player_button_click)

    # Create button container widget
    button_container = widgets.VBox([manager_button, player_button])

    # Create screen container widget
    screen_container = widgets.VBox([title_widget, subtitle_widget, button_container])

    return screen_container

def createPopup():
    overlay = widgets.Output(layout={'border': '1px solid black', 'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px'})

    def handle_exit_button_click(b):
        overlay.close()
        display(createWelcomeScreen())

    def handle_submit_button_click(b):
        choice = password_input.value
        if choice != "shablool":
            error_label.value = "The choice is incorrect, please try again"
        else:
            overlay.close()
            display(createManagerScreen())

    with overlay:
        exit_button = widgets.Button(description="X", button_style="danger")
        exit_button.on_click(handle_exit_button_click)
        password_input = widgets.Password(placeholder="Enter password")
        error_label = widgets.HTML(value="", layout={'color': 'red'})
        submit_button = widgets.Button(description="Submit")
        submit_button.on_click(handle_submit_button_click)

        content = widgets.VBox([widgets.HBox([exit_button], layout={'justify_content': 'flex-end'}), 
                               password_input, error_label, submit_button])
        display(content)
      
    return overlay

overlay = None

def handle_exit_button_click(b):
  overlay.close()
  display(createManagerScreen())

def handle_add_quiz_button_click(b):
    overlay.clear_output()  # Clear the output instead of closing it
    createAddQuiz(handle_exit_button_click)  # Pass the exit callback

def handle_remove_quiz_button_click(b):
    overlay.clear_output()  # Clear the output instead of closing it
    createRemoveQuiz()  # Display the remove quiz screen

def createAddQuiz(exit_callback):
    global overlay  # Declare overlay as a global variable

    def handle_exit_button_click_to_manager(b):
      form.close()
      overlay.close()
      display(createManagerScreen())

    form = widgets.VBox()

    exit_div = widgets.VBox(layout={'display': 'flex', 'justify_content': 'flex-end'})
    exit_button = widgets.Button(description="X")
    exit_button.button_style = "danger"
    exit_button.on_click(handle_exit_button_click_to_manager)
    exit_div.children = [exit_button]
    form.children = [exit_div]

    question_input = widgets.Text(placeholder="Question")
    option1_input = widgets.Text(placeholder="Option 1")
    option2_input = widgets.Text(placeholder="Option 2")
    option3_input = widgets.Text(placeholder="Option 3")
    option4_input = widgets.Text(placeholder="Option 4")
    correct_input = widgets.Dropdown(options=["Option 1", "Option 2", "Option 3", "Option 4"])
    submit_button = widgets.Button(description="Submit")
    error_label = widgets.HTML(value="One of the fields is empty.", layout={'display': 'none', 'color': 'red'})
    success_label = widgets.HTML(value="The quiz added successfully!", layout={'display': 'none', 'color': 'green'})
    form.children += (question_input, option1_input, option2_input, option3_input, option4_input, correct_input,
                      submit_button, error_label, success_label)

    def handle_submit_button_click(b):
        question = question_input.value
        option1 = option1_input.value
        option2 = option2_input.value
        option3 = option3_input.value
        option4 = option4_input.value
        correct = correct_input.value

        if not question or not option1 or not option2 or not option3 or not option4 or not correct:
            error_label.layout.display = 'block'
            success_label.layout.display = 'none'
        else:
            # Check if the question already exists
            existing_questions = FBconn.get('/Questions', None)  # Assuming 'questions' is the path to the question data in your Firebase database

            if existing_questions:
                existing_questions = existing_questions.values()
                if any(q['question'] == question for q in existing_questions):
                    error_label.value = "The question already exists."
                    error_label.layout.display = 'block'
                    success_label.layout.display = 'none'
                    return

            # Add the quiz to the database
            data = {
                'question': question,
                'options': [option1, option2, option3, option4],
                'correctOption': correct
            }
            result = FBconn.post('/Questions', data)  # Assuming 'questions' is the path to the question data in your Firebase database

            if result:
                error_label.layout.display = 'none'
                success_label.layout.display = 'block'
                question_input.value = ""
                option1_input.value = ""
                option2_input.value = ""
                option3_input.value = ""
                option4_input.value = ""
            else:
                error_label.value = "Failed to add the quiz to the database. Please try again."



    submit_button.on_click(handle_submit_button_click)

    overlay.close()
    display(form)

def createRemoveQuiz():
    def removeQuestion(questionId):
        FBconn.delete(f'/Questions/{questionId}', None)
        question_element = next(
            (c for c in remove_quiz_container.children if c.question_id == questionId), None
        )
        if question_element:
            remove_quiz_container.children = tuple(
                c for c in remove_quiz_container.children if c != question_element
            )

    def handle_exit_button_click_to_manager(b):
        remove_quiz_container.close()
        exit_button.close()
        overlay.close()
        display(createManagerScreen())

    remove_quiz_container = widgets.VBox()
    question_containers = []

    questions = FBconn.get('/Questions', None)
    if questions:
        for question_id, question_data in questions.items():
            question_container = widgets.HBox()
            question_text = widgets.HTML(f"<p>{question_data['question']}</p>")
            remove_button = widgets.Button(description="X", button_style="danger")
            remove_button.on_click(lambda b, q_id=question_id: removeQuestion(q_id))
            question_container.children = [question_text, remove_button]
            question_container.question_id = question_id
            question_containers.append(question_container)

    exit_button = widgets.Button(description="Exit", button_style="danger")
    exit_button.on_click(handle_exit_button_click_to_manager)
    remove_quiz_container.children = question_containers + [exit_button]

    overlay.close()
    display(remove_quiz_container)


def createManagerScreen():
    global overlay  # Declare overlay as a global variable

    # Create the overlay widget
    overlay = widgets.Output(layout={'border': '1px solid black', 'background-color': 'rgba(0, 0, 0, 0.5)', 'padding': '20px'})



    with overlay:
        def handle_exit_button_click_to_popup(b):
          overlay.close()
          display(createPopup())

        # Create the manager screen widget
        manager_screen = widgets.VBox()

        # Create the exit button
        exit_button = widgets.Button(description="X")
        exit_button.button_style = "danger"
        exit_button.on_click(handle_exit_button_click_to_popup)

        # Create the manager title
        manager_title = widgets.HTML("<h1>Manager Screen</h1>")

        # Create the add quiz button
        add_quiz_button = widgets.Button(description="Add Quiz")
        add_quiz_button.on_click(handle_add_quiz_button_click)
        add_quiz_button.style.button_color = "lightblue"

        # Create the remove quiz button
        remove_quiz_button = widgets.Button(description="Remove Quiz")
        remove_quiz_button.on_click(handle_remove_quiz_button_click)
        remove_quiz_button.style.button_color = "lightblue"

        # Arrange the widgets
        manager_screen.children = [exit_button, manager_title, add_quiz_button, remove_quiz_button]
        manager_screen.layout = {'border': '1px solid black', 'padding': '10px', 'margin': '10px'}

        # Display the manager screen widget
        display(manager_screen)

    return overlay

screen_container = createWelcomeScreen()
display(screen_container)


Output(layout=Layout(border='1px solid black', padding='20px'))

Output(layout=Layout(border='1px solid black', padding='20px'))

Output(layout=Layout(border='1px solid black', padding='20px'))

Output(layout=Layout(border='1px solid black', padding='20px'))

Output(layout=Layout(border='1px solid black', padding='20px'))